# High-Accuracy Handwritten Character Recognition using EMNIST
This notebook builds a robust CNN model using the EMNIST dataset, which is a larger and more diverse dataset than the previous one. This should result in significantly higher accuracy.

### For Google Colab Users
To speed up training, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 2. Load and Preprocess the EMNIST Dataset

In [ ]:
# Load the EMNIST 'byclass' dataset
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/byclass',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Define class names (62 classes: 0-9, A-Z, a-z)
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# Preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    # EMNIST images are rotated and flipped, we need to correct for that
    image = tf.transpose(image, perm=[1, 0, 2])
    return image, label

# Apply preprocessing
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch the datasets
BATCH_SIZE = 128
ds_train = ds_train.cache().shuffle(ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

print(f'Training dataset specs: {ds_train}')
print(f'Test dataset specs: {ds_test}')

## 3. Visualize Sample Images

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().squeeze(), cmap='gray')
        plt.title(class_names[labels[i]])
        plt.axis('off')
plt.show()

## 4. Build a More Powerful CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25), # Add dropout for regularization
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5), # Add more dropout
    Dense(len(class_names), activation='softmax') # 62 output classes
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy', # Use sparse CE because labels are integers
              metrics=['accuracy'])

model.summary()

## 5. Train the Model

In [ ]:
# No data augmentation is needed here as tf.data pipeline is more efficient
# and EMNIST is already very diverse.
history = model.fit(ds_train, epochs=20, validation_data=ds_test)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

## 6. Evaluate the Model

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(ds_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Get predictions for the entire test set
y_pred_probs = model.predict(ds_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true = np.concatenate([y for x, y in ds_test], axis=0)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(20, 20))
sns.heatmap(conf_matrix, annot=False, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Annotations disabled for clarity)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Classification Report
print('
Classification Report:')
print(classification_report(y_true, y_pred_classes, target_names=class_names))

## 7. Save the Final, Accurate Model

In [ ]:
model.save('htr_model.h5')
print('Model saved as htr_model.h5. You can now use this with your Flask app.')